In [1]:
from bs4 import BeautifulSoup
import re
import string
def cleaner(str):
    soup = BeautifulSoup(str)
    str1 = soup.get_text()
    str1 = str1.replace('\\n', ' ')
    str2 = str1.replace("\\","")
    str3 = str2.replace("("," ")
    str4 = str3.replace(")"," ")
    str5 = re.sub("[0-9]|\.|{|}|\^|;|=|/|\"" , " ", str4)
    return str5

In [2]:
# import csv
# csvFile = open('./data2/questions.csv', 'rb')
# spamreader = csv.reader(csvFile, delimiter=',', quotechar='"')
# # df = pandas.read_csv( , header = 1 , )
# docs = []
# labels = []
# counter = 0
# import sys
# sys.stdout = open('./data2/out.csv', 'w')
# writeFile = open('', 'w')
# writer = csv.writer(writeFile)

In [3]:
# for rows in spamreader:
#     counter = counter + 1
#     str1 = cleaner(rows[3])
#     str2 = cleaner(rows[4])
#     outstr = str1 + str2
#     outstr = ''.join(x for x in outstr if x in string.printable)
# #     outstr.decode('utf-8','ignore').encode("utf-8")
#     labl = cleaner(rows[20]).rstrip()
#     if (labl == "P"):
#         print("\"") , 
#         print( outstr) ,
#         print("\"") , 
#         print(" , "),
#         print(0)
#     if (labl == "C"):
#         print("\"") , 
#         print( outstr) ,
#         print("\"") , 
#         print(" , "),
#         print(1)
#     if (labl == "M"):
#         print("\"") , 
#         print( outstr) ,
#         print("\"") , 
#         print(" , "),
#         print(2)
    
    


In [4]:
import pandas


In [5]:
df = pandas.read_csv('./data2/out.csv',header = None)
d = df.to_dict()





In [11]:
import pandas
import os
import sys
import numpy as np
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.initializers import Constant

Using TensorFlow backend.


In [12]:
MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

BASE_DIR = '../'
GLOVE_DIR = os.path.join(BASE_DIR, 'glove.6B')

print('Indexing word vectors.')

embeddings_index = {}
with open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt')) as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

# second, prepare text samples and their labels
print('Processing text dataset')


Indexing word vectors.
Found 400000 word vectors.
Processing text dataset


In [30]:
docs=[]
labels = []

for value in d[0]:
    docs.append(d[0][value])
    labels.append(d[1][value])

print(labels)

[0, 1, 0, 0, 1, 1, 2, 1, 1, 1, 1, 0, 0, 0, 1, 2, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 2, 2, 0, 2, 1, 0, 1, 1, 0, 1, 1, 2, 0, 0, 0, 2, 2, 1, 0, 1, 0, 1, 0, 0, 0, 2, 2, 2, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 2, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 2, 2, 0, 2, 0, 0, 1, 0, 1, 0, 1, 2, 1, 1, 0, 1, 0, 1, 2, 1, 1, 2, 0, 0, 1, 0, 0, 2, 1, 1, 0, 1, 2, 1, 1, 0, 1, 1, 0, 1, 1, 0, 2, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 2, 0, 1, 1, 0, 0, 1, 1, 1, 2, 1, 1, 0, 0, 0, 0, 0, 2, 0, 1, 1, 2, 0, 2, 0, 2, 0, 0, 2, 0, 2, 0, 2, 0, 1, 1, 0, 1, 2, 1, 1, 0, 0, 0, 1, 1, 2, 0, 1, 1, 1, 1, 0, 2, 0, 1, 2, 1, 0, 1, 2, 0, 1, 1, 1, 0, 2, 0, 0, 2, 0, 1, 1, 0, 1, 1, 0, 2, 2, 2, 2, 1, 1, 0, 1, 0, 0, 0, 1, 2, 1, 1, 2, 0, 0, 1, 2, 0, 2, 2, 1, 0, 1, 2, 2, 1, 0, 0, 1, 2, 1, 2, 2, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 2, 0, 0, 1, 0, 2, 0, 1, 1, 2, 0, 1, 1, 0, 2, 2, 0, 2, 1, 2, 1, 0, 1, 0, 0, 1, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 1, 0, 1, 1, 2, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 2, 0, 0, 0, 2, 1, 1, 1, 0, 1, 2, 0, 1, 1, 1, 0, 0, 0, 0, 

In [32]:
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(docs)
sequences = tokenizer.texts_to_sequences(docs)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]

print('Preparing embedding matrix.')

# prepare embedding matrix
num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i > MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
        embedding_matrix[i] = np.random.random_sample((100))


Found 15666 unique tokens.
('Shape of data tensor:', (21309, 1000))
('Shape of label tensor:', (21309, 3, 2))
Preparing embedding matrix.


In [33]:
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

In [34]:
print('Training model.')

# train a 1D convnet with global maxpooling
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = GlobalMaxPooling1D()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(3, activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

print(model.summary())


model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          validation_data=(x_val, y_val))


Training model.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1000, 100)         1566700   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 996, 128)          64128     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 199, 128)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 195, 128)          82048     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 39, 128)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 35, 128)           82048

ValueError: Error when checking target: expected dense_2 to have 2 dimensions, but got array with shape (17048, 3, 2)